# Processing Time Analysis
This notebook allows you to select and visualize processing time and read time data from marker location files.

In [1]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display
import os
from pathlib import Path

In [2]:
# Base directory for data
base_dir = r"C:\Users\kankean.kandasamy\Repo\KIM-QA-Analysis\Lyrebird Data\2025-10-28"

# Find all .txt files in subdirectories
def find_data_files(base_path):
    files = []
    for root, dirs, filenames in os.walk(base_path):
        for filename in filenames:
            if filename.endswith('.txt') and 'MarkerLocations' in filename:
                full_path = os.path.join(root, filename)
                relative_path = os.path.relpath(full_path, base_path)
                files.append((relative_path, full_path))
    return sorted(files)

data_files = find_data_files(base_dir)
print(f"Found {len(data_files)} data files")

Found 10 data files


In [ ]:
# Create dropdown widget for file selection
file_dropdown = widgets.Dropdown(
    options=[(name, path) for name, path in data_files],
    description='Select File:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

# Create radio buttons for x-axis selection
xaxis_radio = widgets.RadioButtons(
    options=['Frame No', 'Time (sec)'],
    value='Time (sec)',
    description='X-Axis:',
    disabled=False
)

# Create button to trigger plot
plot_button = widgets.Button(
    description='Plot Data',
    button_style='primary',
    tooltip='Click to plot the selected file',
    icon='chart-line'
)

output = widgets.Output()

display(file_dropdown)
display(xaxis_radio)
display(plot_button)
display(output)

Dropdown(description='Select File:', layout=Layout(width='80%'), options=(('Case1 - iso1\\MarkerLocationsGA_Co…

RadioButtons(description='X-Axis:', index=1, options=('Frame No', 'Time (sec)'), value='Time (sec)')

Button(button_style='primary', description='Plot Data', icon='chart-line', style=ButtonStyle(), tooltip='Click…

Output()

In [4]:
def plot_data(file_path, x_axis_col):
    """Read data file and create interactive plots"""
    try:
        # Read the CSV file
        df = pd.read_csv(file_path, skipinitialspace=True)
        
        # Clean column names (remove leading/trailing spaces)
        df.columns = df.columns.str.strip()
        
        # Extract relevant columns
        x_data = df[x_axis_col]
        processing_time = df['Processing Time (ms)']
        read_time = df['Read Time (ms)']
        
        # Create subplots with 2 rows
        fig = make_subplots(
            rows=2, cols=1,
            subplot_titles=('Processing Time', 'Read Time'),
            vertical_spacing=0.12,
            shared_xaxes=True
        )
        
        # Add Processing Time trace (top panel)
        fig.add_trace(
            go.Scatter(
                x=x_data,
                y=processing_time,
                mode='lines+markers',
                name='Processing Time',
                line=dict(color='#1f77b4', width=2),
                marker=dict(size=4),
                hovertemplate=f'{x_axis_col}: %{{x}}<br>Processing Time: %{{y:.2f}} ms<extra></extra>'
            ),
            row=1, col=1
        )
        
        # Add Read Time trace (bottom panel)
        fig.add_trace(
            go.Scatter(
                x=x_data,
                y=read_time,
                mode='lines+markers',
                name='Read Time',
                line=dict(color='#ff7f0e', width=2),
                marker=dict(size=4),
                hovertemplate=f'{x_axis_col}: %{{x}}<br>Read Time: %{{y:.2f}} ms<extra></extra>'
            ),
            row=2, col=1
        )
        
        # Update axes labels
        fig.update_xaxes(title_text=x_axis_col, row=2, col=1)
        fig.update_yaxes(title_text='Processing Time (ms)', row=1, col=1)
        fig.update_yaxes(title_text='Read Time (ms)', row=2, col=1)
        
        # Update layout for better interactivity
        fig.update_layout(
            height=800,
            showlegend=True,
            hovermode='x unified',
            title=dict(
                text=f'<b>{os.path.basename(file_path)}</b>',
                x=0.5,
                xanchor='center'
            ),
            font=dict(size=12)
        )
        
        # Enable pan and zoom
        fig.update_xaxes(rangeslider_visible=False)
        
        return fig, df
        
    except Exception as e:
        print(f"Error reading file: {e}")
        return None, None

def on_plot_button_clicked(b):
    """Handle plot button click"""
    with output:
        output.clear_output(wait=True)
        selected_file = file_dropdown.value
        x_axis = xaxis_radio.value
        
        if selected_file:
            print(f"Loading: {os.path.basename(selected_file)}")
            fig, df = plot_data(selected_file, x_axis)
            
            if fig is not None:
                fig.show()
                print(f"\nData Summary:")
                print(f"Total Frames: {len(df)}")
                print(f"\nProcessing Time (ms):")
                print(f"  Mean: {df['Processing Time (ms)'].mean():.2f}")
                print(f"  Median: {df['Processing Time (ms)'].median():.2f}")
                print(f"  Min: {df['Processing Time (ms)'].min():.2f}")
                print(f"  Max: {df['Processing Time (ms)'].max():.2f}")
                print(f"\nRead Time (ms):")
                print(f"  Mean: {df['Read Time (ms)'].mean():.2f}")
                print(f"  Median: {df['Read Time (ms)'].median():.2f}")
                print(f"  Min: {df['Read Time (ms)'].min():.2f}")
                print(f"  Max: {df['Read Time (ms)'].max():.2f}")
        else:
            print("Please select a file to plot.")

plot_button.on_click(on_plot_button_clicked)

## Instructions
1. Select a data file from the dropdown menu
2. Choose the X-axis variable (Frame No or Time)
3. Click 'Plot Data' to visualize
4. Use Plotly's interactive features:
   - **Pan**: Click and drag
   - **Zoom**: Use scroll wheel or click and drag to select area
   - **Box/Lasso Select**: Click mode icons in toolbar
   - **Reset**: Double-click the plot
   - **Hover**: Move mouse over data points for details